In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances_chunked
import sqlalchemy as sa

engine = sa.create_engine('sqlite:///data/distances.db') 

In [2]:
df = pd.read_csv('data/year_prediction.csv')
df

,label,TimbreAvg1,TimbreAvg2,TimbreAvg3,TimbreAvg4,TimbreAvg5,TimbreAvg6,TimbreAvg7,TimbreAvg8,TimbreAvg9,...,TimbreCovariance69,TimbreCovariance70,TimbreCovariance71,TimbreCovariance72,TimbreCovariance73,TimbreCovariance74,TimbreCovariance75,TimbreCovariance76,TimbreCovariance77,TimbreCovariance78
0,2001,49.94357,21.47114,73.07750,8.74861,-17.40628,-13.09905,-25.01202,-12.23257,7.83089,...,13.01620,-54.40548,58.99367,15.37344,1.11144,-23.08793,68.40795,-1.82223,-27.46348,2.26327
1,2001,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,...,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
2,2001,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,...,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
3,2001,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,...,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
4,2001,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,...,9.92661,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515340,2006,51.28467,45.88068,22.19582,-5.53319,-3.61835,-16.36914,2.12652,5.18160,-8.66890,...,4.81440,-3.75991,-30.92584,26.33968,-5.03390,21.86037,-142.29410,3.42901,-41.14721,-15.46052
515341,2006,49.87870,37.93125,18.65987,-3.63581,-27.75665,-18.52988,7.76108,3.56109,-2.50351,...,32.38589,-32.75535,-61.05473,56.65182,15.29965,95.88193,-10.63242,12.96552,92.11633,10.88815
515342,2006,45.12852,12.65758,-38.72018,8.80882,-29.29985,-2.28706,-18.40424,-22.28726,-4.52429,...,-18.73598,-71.15954,-123.98443,121.26989,10.89629,34.62409,-248.61020,-6.07171,53.96319,-8.09364
515343,2006,44.16614,32.38368,-3.34971,-2.49165,-19.59278,-18.67098,8.78428,4.02039,-12.01230,...,67.16763,282.77624,-4.63677,144.00125,21.62652,-29.72432,71.47198,20.32240,14.83107,39.74909


In [3]:
def convert_array_to_ints(a: np.array, n=2) -> int:
    a *= (n * 10)
    return a.astype(int)

In [4]:
class DataFrameChunks:
    def __init__(self, generator):
        self.generator = generator
        self.n = 0
        
    def __next__(self) -> pd.DataFrame:
        chunk = next(generator)
        new_n = self.n + len(chunk)-1
        index = range(self.n, new_n+1)
        self.n = new_n + 1
        return pd.DataFrame(chunk, index=index)
    
    def __iter__(self):
        return self

In [8]:
def top_scores(row, row_date, dates, n=10):
    df = pd.DataFrame({
        'x_index': row.name,
        'score': row,
        'y_date':  dates
    })
    df = df[df['y_date'] < row_date]
    df.index.name = 'y_index'
    return df.sort_values('score').head(n).reset_index()

In [9]:
from sqlalchemy import Table, Column, Integer, String, MetaData, Float
meta = MetaData()

students = Table(
    'top_10', meta, 
    Column('x_index', Integer, primary_key=True), 
    Column('y_index', Integer, primary_key=True), 
    Column('score', Float), 
    Column('y_date', Integer), 
)

if 'top_10' not in engine.table_names():
    meta.create_all(engine)

/var/folders/fs/k9yjsldj2438vskn8pmgmh4h0000gn/T/ipykernel_25353/2958350078.py:12: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  if 'top_10' not in engine.table_names():


In [10]:
generator = pairwise_distances_chunked(df.drop('label', axis=1), n_jobs=8, )
df_generator = DataFrameChunks(generator)

for chunk_i, chunk in enumerate(df_generator):
    for row_i, row in chunk.iterrows():
        scores = top_scores(row, df.iloc[row_i]['label'], df['label'])
        scores.to_sql('top_10', engine, if_exists='append', index=False)
    print(row_i, 'done')

259 done
519 done
779 done
1039 done
1299 done
1559 done
1819 done
2079 done
2339 done
2599 done
2859 done
3119 done
3379 done
3639 done
3899 done
4159 done
4419 done
4679 done
4939 done
5199 done
5459 done
5719 done
5979 done
6239 done
6499 done
6759 done
7019 done
7279 done
7539 done
7799 done
8059 done
8319 done
8579 done
8839 done
9099 done
9359 done
9619 done
9879 done
10139 done
10399 done
10659 done
10919 done
11179 done
11439 done
11699 done
11959 done
12219 done
12479 done
12739 done
12999 done
13259 done
13519 done
13779 done
14039 done
14299 done
14559 done
14819 done
15079 done
15339 done
15599 done
15859 done
16119 done
16379 done
16639 done
16899 done
17159 done
17419 done
17679 done
17939 done
18199 done
18459 done
18719 done
18979 done
19239 done
19499 done
19759 done
20019 done
20279 done
20539 done
20799 done
21059 done
21319 done
21579 done
21839 done
22099 done
22359 done
22619 done
22879 done
23139 done
23399 done
23659 done
23919 done
24179 done
24439 done
24699 d

187199 done
187459 done
187719 done
187979 done
188239 done
188499 done
188759 done
189019 done
189279 done
189539 done
189799 done
190059 done
190319 done
190579 done
190839 done
191099 done
191359 done
191619 done
191879 done
192139 done
192399 done
192659 done
192919 done
193179 done
193439 done
193699 done
193959 done
194219 done
194479 done
194739 done
194999 done
195259 done
195519 done
195779 done
196039 done
196299 done
196559 done
196819 done
197079 done
197339 done
197599 done
197859 done
198119 done
198379 done
198639 done
198899 done
199159 done
199419 done
199679 done
199939 done
200199 done
200459 done
200719 done
200979 done
201239 done
201499 done
201759 done
202019 done
202279 done
202539 done
202799 done
203059 done
203319 done
203579 done
203839 done
204099 done
204359 done
204619 done
204879 done
205139 done
205399 done
205659 done
205919 done
206179 done
206439 done
206699 done
206959 done
207219 done
207479 done
207739 done
207999 done
208259 done
208519 done
2087

364779 done
365039 done
365299 done
365559 done
365819 done
366079 done
366339 done
366599 done
366859 done
367119 done
367379 done
367639 done
367899 done
368159 done
368419 done
368679 done
368939 done
369199 done
369459 done
369719 done
369979 done
370239 done
370499 done
370759 done
371019 done
371279 done
371539 done
371799 done
372059 done
372319 done
372579 done
372839 done
373099 done
373359 done
373619 done
373879 done
374139 done
374399 done
374659 done
374919 done
375179 done
375439 done
375699 done
375959 done
376219 done
376479 done
376739 done
376999 done
377259 done
377519 done
377779 done
378039 done
378299 done
378559 done
378819 done
379079 done
379339 done
379599 done
379859 done
380119 done
380379 done
380639 done
380899 done
381159 done
381419 done
381679 done
381939 done
382199 done
382459 done
382719 done
382979 done
383239 done
383499 done
383759 done
384019 done
384279 done
384539 done
384799 done
385059 done
385319 done
385579 done
385839 done
386099 done
3863